<a href="https://colab.research.google.com/github/Praveenraj59/Next-Word-Prediction-in-NLP/blob/main/Next_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re


In [23]:
def file_to_sentence_list(file_path):
	with open(file_path, 'r') as file:
		text = file.read()

	# Splitting the text into sentences using
	# delimiters like '.', '?', and '!'
	sentences = [sentence.strip() for sentence in re.split(
		r'(?<=[.!?])\s+', text) if sentence.strip()]

	return sentences

file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

# Create input sequences
input_sequences = []
for line in text_data:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
	input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)


In [18]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,
					input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy',
			optimizer='adam', metrics=['accuracy'])


In [25]:
# Train the model
model.fit(X, y, epochs=100, verbose=1)


Epoch 1/100
51/51 [==============================] - 3s 56ms/step - loss: 0.6633 - accuracy: 0.9054
Epoch 2/100
51/51 [==============================] - 2s 42ms/step - loss: 0.6448 - accuracy: 0.9079
Epoch 3/100
51/51 [==============================] - 2s 43ms/step - loss: 0.6259 - accuracy: 0.9103
Epoch 4/100
51/51 [==============================] - 2s 42ms/step - loss: 0.6079 - accuracy: 0.9134
Epoch 5/100
51/51 [==============================] - 2s 44ms/step - loss: 0.5950 - accuracy: 0.9195
Epoch 6/100
51/51 [==============================] - 4s 88ms/step - loss: 0.5761 - accuracy: 0.9208
Epoch 7/100
51/51 [==============================] - 2s 43ms/step - loss: 0.5650 - accuracy: 0.9195
Epoch 8/100
51/51 [==============================] - 2s 42ms/step - loss: 0.5502 - accuracy: 0.9269
Epoch 9/100
51/51 [==============================] - 2s 43ms/step - loss: 0.5348 - accuracy: 0.9238
Epoch 10/100
51/51 [==============================] - 2s 43ms/step - loss: 0.5215 - accuracy: 0.9281

In [26]:
# Generate next word predictions
seed_text = "Pizza have different "
next_words = 5

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)


1/1 [==============================] - 0s 23ms/step
Next predicted words: Pizza have different  become a symbol of comfort
